## working with WRF_CHEM:compiling and install prep-chem-src

- to isntall prep-chem-src, it requiers to install HDF5,ZLIB and NETCDF
- based on the readme of PREP_CHEM_SRC as follows
```bash
Third Party Software Requirements
 1. ZLIB 1.2.5(libz.a) or later  distribution.  You may download the software
    from the http://www.zlib.net/ site.
 2. HDF5-1.8.8(libhdf5_fortran.a, libhdf5hl_fortran.a) or later  distribution.  You may download the software
    from http://www.hdfgroup.org/HDF5/release/obtain5.html.    
 3. NetCDF 4.1.1 (libnetcdf.a).  You may download the software
    from http://www.unidata.ucar.edu/downloads/netcdf/netcdf-4_0_1/index.jsp
Configuring/Installing HDF 5
When compiling PREP-CHEM-SRC codes on a Linux system using the PGI (Intel)
compiler, make sure the netCDF and HDF* library has been installed using the same PGI (Intel) compiler.
FC=pgf90 CC=pgcc CXX=pgc++ ./configure  \
	--prefix=/path_libs/hdf5-1.8.8 \
	--with-zlib=/path_libs/zlib-1.2.5  \
	--disable-shared \
	--enable-fortran
make
make check
make install
make check-install
INSTALL NETCDF
FC=pgf90 CC=pgcc CXX=pgc++ ./configure --prefix=/path-netcdf-4.1.1
make
make check
make install
```
- so first step installing HDF5, checked current coputer and tryed to run ```dpkg-query --show hdf5``` gives nill
- so has to compile from source, it is downloaded from [here](http://www.hdfgroup.org/ftp/HDF5/releases/hdf5-1.8.8/src/hdf5-1.8.8.tar.gz)
- after unzipping, went inside the dicertoty and done the stpe as told in the README, 
```bash
FC=pgf90 CC=pgcc CXX=pgc++ ./configure  \
	--prefix=/path_libs/hdf5-1.8.8 \
	--with-zlib=/path_libs/zlib-1.2.5  \
	--disable-shared \
	--enable-fortran
```
1. but this step exited with errors
- then found out that the pgf90 is a commercial compiler and so used the gfortran available in the ubuntu 12.04 machine so the comamnd becomes
```bash
FC=gfortran CC=gcc CXX=gc++ ./configure  --prefix=/path_libs/hdf5-1.8.8 --with-zlib=/path_libs/zlib-1.2.5  --disable-shared --enable-fortran
```
- after running it shows something something and went to next step ```make```, it gives and run for sioometimes with some words and then next step
- in doing ```make chekc``` it gives three errors, and it was negletced and went for next step
- in this instead of doing ```make install```, made a change of routing, make isntall delivbratley add new files where were it do so and for removing
  it is impossible taks to read and remove the file, more details [here](http://superuser.com/questions/264601/removing-installations-done-by-make-install)
- so followed and executed this ```checkinstall``` but asks to uisnatll checkinstall ```sudo apt-get isntall checkimnstall```
- aftyer sthis follwoed ```sudo checkinstall``` and it asks some questions aenetered all and findlly ti gives godd news of 
```bash
 Done. The new package has been installed and saved to
 /home/swl-sacon-dst/wrf/WRF341/Emission/hdf5-1.8.8/hdf5_1.8.8-1_amd64.deb
 You can remove it from your system anytime using: 
      dpkg -r hdf5
```
- so now trying the comment ``` dpkg-query --show hdf5``` giives ```hdf5	1.8.8-1```

### insall zlib 1.2.5

- followed as per the README but with gfortran and checkinstall
```bash
FC=gfortran CC=gcc ./configure --prefix=/path-zlib-1.2.5
make
make check
make checkinstall
```
- retunred as quickly the good news of isntallation success

### intsall netcdf4.1.1

- downloaded netcdf4.1.1 and went and run as per the README but with gfortran and checkinstall but returned wrror saying ```undefined open_mpi_char```
- so installing openmpi1.6.5, openmpi currently is 1.5 and retired it seems, so trying with source compilation as per [this](http://www.open-mpi.org/community/lists/users/2012/07/19813.php)
- the code followed is
```bash
FC=gfortran CC=gcc CXX=gc++ ./configure --enable-fortran
```
- but gives error of ```configure: error: C and C++ compilers are not link compatible.  Can not continue.```
- so run the ```./configure``` with pretext, let it take waht ever availble in computer! and it completet with out any error
-  next made netcdf install with 
```bash
FC=gfortran CC=gcc CXX=gc++ ./configure --prefix=/path-netcdf-4.1.1
make
``` but exited with same error of ```undefined reference to `ompi_mpi_char'```
- googling shows this [page](http://www.unidata.ucar.edu/mailing_lists/archives/netcdfgroup/2010/msg00455.html) saying HDF5 has to be installed with ```--enable parallell```
- so removing HDF5 earlier made, ```sudo dpkg -r hdf5```
- redoing it with 
```bash
FC=gfortran CC=gcc CXX=gc++ ./configure  --prefix=/path_libs/hdf5-1.8.8 --enable-parallel --with-zlib=/path_libs/zlib-1.2.5  --disable-shared --enable-fortran
make
```
- but netcdf still giving the same error of ```undefined reference to `ompi_mpi_char'```
- so doing configure without any options, it also giving the same error
- so with the available netcdf(I am figuring out how this netcdf was installed) in the computer, used the location of netcdf and run prep-chem-src, but it giving huge chunk of error starting with non availability of hdf.
- In this way as told in README, it gets installed in ```/path_libs```
- some useful comments, to view the location of installed packages ```dpkg -L netcdf```, to view the compilers in the computer ```dpkg --list | grep compiler```, to view environemtn realted with netcdf ```printenv  | grep NETCDF ```
- A fresh try was made after a cipule of days, in which googleing was made with compilation error given, one such error is ``` undefined reference to `__netcdf_MOD_nf90_open'```, this search fetched to view the nc-config to know about the lib forlder and include folder of installed compiled netcdf, it seems, for installing IRIS python package, netcdf 4.3.2 was isnatlled and it was located in /usr/local ptth, the command ```nc-config --all``` gives all like this info. 
- so the file in prpe_chem_src folder bin/build ```include.mk.opt.gfortran``` is edited to change this new info, but compilation failed with similar error. more search on this regard saying that one needs to include a new netcdff with double f has to be include, this clue is given in [this](http://www.yambo-code.org/forum/viewtopic.php?f=1&t=755) , [this](https://www.myroms.org/forum/viewtopic.php?f=31&t=1414) and [this](http://ubuntuforums.org/archive/index.php/t-1033570.html), more over directly solving this problem for prep chem src, there is an arabic language web page showing that the file ```include.mk.opt.gfortran``` has to be edited and the line asking for netcdf library has to be ```NETCDF_LIBS=-L$(NETCDF)/lib -lnetcdf -lnetcdff``` instead of ```NETCDF_LIBS=-L$(NETCDF)/lib -lnetcdf```. later it is found the the tutorial for nepal WRF-CHEM aslo saying this to be changed, hrrr. "half a hour wrf manual reading is better than one week google search" 
- with this changes the prep_chem_src is get compiled with good news of but with warnings of ```/usr/bin/ld: warning: libhdf5.so.6, needed by /usr/lib/libnetcdf.so.6, may conflict with libhdf5.so.8```.
```bash
Finished building === ../prep_chem_sources_RADM_WRF_FIM.exe
```
